# 패키지 로드

In [1]:
from itertools import islice # for print head
import re # 정규표현식 모듈
from pandas import DataFrame as df # dataframe 을 활용하기 위함
import pandas as pd
from operator import eq
import math
import numpy as np

# 2. TF-IDF of documents

각 챕터별로 문서의 TF-IDF 를 구함

In [2]:
# preprocessed 된 데이터 로드
chapterspath = "data/prerocessed.xlsx"
tokens = pd.read_excel(chapterspath)

In [3]:
tokens

,chapter,token
0,2,mr
1,2,bennet
2,2,among
3,2,earliest
4,2,waited
5,2,mr
6,2,bingley
7,2,always
8,2,intended
9,2,visit


## 2.1 Term Frequency (TF)

### 2.1.1 Simple word counts (TF)

In [4]:
# 중복제거함으로서 챕터별로 존재하는 단어의 종류와 중복된 개수를 선택. 이것이 TF가 됨
tfs = pd.DataFrame(tokens, columns=['chapter','token'])
tfs = tfs.groupby(tfs.columns.tolist(), as_index=False).size().reset_index().rename(columns={0:'count'})

In [5]:
# 문서에 대한 각 단어의 tf를 밖으로 빼냄
import pandas as pd
writer = pd.ExcelWriter('data/TF.xlsx')
tfs.to_excel(writer,'Sheet1')
writer.save()

### 2.1.1 Normalized TF for Document

In [6]:
# 각 단어의 빈도 / 챕터 내 전체 빈도
# 챕터 내 빈도
tf_sums = tfs.groupby(['chapter'], as_index=False)["count"].sum()
tf_sums = tf_sums.rename(columns={'count':'sum'})
tf_sums

,chapter,sum
0,2,346
1,3,800
2,4,470
3,5,445
4,6,1051
5,7,923
6,9,809
7,11,743
8,13,744
9,15,815


In [7]:
tfs_norm = tfs.set_index('chapter').join(tf_sums.set_index('chapter'))
tfs_norm["tf_norm"] = tfs_norm["count"] / tfs_norm["sum"]
tfs_norm = tfs_norm.reset_index()

In [8]:
# 문서에 대한 각 단어의 normalized tf를 밖으로 빼냄
import pandas as pd
writer = pd.ExcelWriter('data/TF_norm.xlsx')
tfs_norm.to_excel(writer,'Sheet1')
writer.save()

## 2.2 Inverse Document Frequency (IDF)

IDF(term) = 1 + loge(Total Number Of Chapters / Number Of Chapters with the term in it)

In [9]:
# 챕터 개수
TotalNumberOfChapters = len(tf_sums)

In [10]:
# 앞에서 구한 tf 에는 단어가 챕터별로 중복없이 들어있음.
# 따라서 각 단어의 단어의 개수를 카운트 하면 각 단어가 들어있는 챕터의 개수를 구할 수 있음.
idf = tfs_norm.groupby(['token'], as_index=False).size().reset_index().rename(columns={0:'numberOfChap'})
idf["idf"] = np.log( TotalNumberOfChapters / idf["numberOfChap"]) + 1

In [11]:
# 문서에 대한 각 단어의 IDF를 밖으로 빼냄
import pandas as pd
writer = pd.ExcelWriter('data/IDF.xlsx')
idf.to_excel(writer,'Sheet1')
writer.save()

## 2.3 TF-IDF of documents

In [12]:
# 각 토큰에 대해서 IDF값을 join. 같은 token끼리 join 해준다. 
# left join
tf_idf = pd.merge(tfs_norm, idf, how='left', on=['token', 'token'])

In [13]:
tf_idf["tf_idf"] = tf_idf["tf_norm"]*tf_idf["idf"]

In [14]:
# 문서에 대한 각 단어의 TF-IDF를 밖으로 빼냄
import pandas as pd
writer = pd.ExcelWriter('data/tf_idf.xlsx')
tf_idf.to_excel(writer,'Sheet1')
writer.save()